# DF

In [ ]:
import marimo as mo
import polars as pl

from r_translation import join

df = join()

# matching and contentious split
What makes a data point contentious is where it has duplicate speciesId.

In [ ]:
matching = df.filter(~pl.col("speciesId").is_duplicated()).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(-1))
    .cast(pl.Int64)
)
contentious = df.filter((pl.col("speciesId").is_duplicated())).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(-1))
    .cast(pl.Int64)
)

unique_contentious = contentious.filter(
    (pl.col("acceptedNameUsageID") == -1)
    & (
        pl.col("matched_taxonID").is_in(
            pl.col("acceptedNameUsageID").implode()
        )
    )
)
# print(
#     unique_contentious.select(
#         "speciesId", "acceptedNameUsageID", "matched_taxonID"
#     ).collect()
# )
unique_contentius_speciesId = (
    unique_contentious.select("speciesId").collect().to_series().implode()
)  # Just the speciesIds
contentious2 = contentious.filter(
    ~pl.col("speciesId").is_in(unique_contentius_speciesId)
)  # Removing...

# print(contentious2.select('speciesId','acceptedNameUsageID','matched_taxonID').collect())

matching = pl.concat(
    [matching, unique_contentious],
)

nomatch lazyframe is for BOS data point that has no current match in gbif data set.

In [ ]:
nomatch = (
    matching.filter(pl.col("matched_taxonID").is_null())
    .with_columns(
        taxonRank=pl.lit("BOSuncornirmedSpecies"),
        taxonomicStatus=pl.lit("BOSunformired"),
        parentNameUsageID=pl.lit(None),
    )
    .collect()
)
nomatch

speciesId,matched_taxonID,acceptedNameUsageID,taxonName,domain,kingdom,phylum,class,subclass,superorder,order,sub-order,infraorder,section,subsection,superfamily,family,subfamily,tribe,genus,subgenus,genericName,specificEpithet,infraspecificEpithet,taxonRank,taxonomicStatus,parentNameUsageID
str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,null
"""A-Anne-Clit-Hirudinea-000001""",null,-1,"""Glossiphoniidae sp. ""1""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Rhynchobdellida""",null,null,null,null,null,"""Glossiphoniidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000002""",null,-1,"""Glossiphoniidae sp. ""2""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Rhynchobdellida""",null,null,null,null,null,"""Glossiphoniidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000003""",null,-1,"""Glossiphoniidae sp. ""3""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Rhynchobdellida""",null,null,null,null,null,"""Glossiphoniidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000004""",null,-1,"""Hirudinidae sp. ""1""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Arhynchobdellida""",null,null,null,null,null,"""Hirudinidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Anne-Clit-Hirudinea-000005""",null,-1,"""Hirudinidae sp. ""2""""",null,null,"""Annelida""","""Clitellata""","""Hirudinea""",null,"""Arhynchobdellida""",null,null,null,null,null,"""Hirudinidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A-Hemi-Enteropneusta-000010""",null,-1,"""Acorn worm sp.""",null,null,"""Hemichordata""","""Enteropneusta""",null,null,"""blank-UnID""",null,null,null,null,null,null,null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Hemi-Enteropneusta-000011""",null,-1,"""Acorn worm sp.""",null,null,"""Hemichordata""","""Enteropneusta""",null,null,"""blank-UnID""",null,null,null,null,null,null,null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null
"""A-Phoronida-000002""",null,-1,"""Phoronida sp.""",null,null,"""Phoronida""","""blank-Phor""",null,null,"""blank-Phor""",null,null,null,null,null,"""Phoronidae""",null,null,null,null,null,null,"""""","""BOSuncornirmedSpecies""","""BOSunformired""",null


In [ ]:
repeated_accepted_taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(pl.col("taxonomicStatus") == pl.lit("accepted"))
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .filter(~pl.col("canonicalName").is_null())
    .filter(pl.col("canonicalName") != "")
    .sort("canonicalName")
    .filter(pl.col("canonicalName").is_duplicated())
)

### repeated_accepted_taxons = RAT

In [ ]:
# repeated_accepted_taxons.collect_schema().keys()

In [ ]:
priority_columns = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
    # "domain",
]

_schema = {
    "feature_that_is_equal_to_canonicalName": pl.String,
    "matches": pl.String,
}
_r = repeated_accepted_taxons.collect()
RAT_interim = pl.DataFrame(schema=_schema)
for _c in priority_columns:
    _a = (
        _r.filter(pl.col("canonicalName") == pl.col(_c))
        .select("canonicalName")
        .unique()
        # .collect()
    )
    if _a.shape[0] != 0:
        _t = _a["canonicalName"].to_list()

        _row = pl.DataFrame(
            data={"feature_that_is_equal_to_canonicalName": _c, "matches": _t},
            schema=_schema,
        )
        RAT_interim = RAT_interim.vstack(_row)
RAT_feats = (
    RAT_interim.group_by("matches")
    .agg(pl.col("feature_that_is_equal_to_canonicalName").str.join(", "))
    .with_columns(
        pl.col("feature_that_is_equal_to_canonicalName").str.split(", ")
    )
    .sort("feature_that_is_equal_to_canonicalName")
)
# RAT_feats.write_csv("RAT_feats.csv")

In [ ]:
priorityFeatures = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
]
pl.Config.set_tbl_cols(1000)

updated_nomatch = pl.LazyFrame()
# all_taxon_data_to_be_selected_from = pl.LazyFrame()
_collected_repeated_taxons = repeated_accepted_taxons.collect()
# for _f in ["family"]:
for _f in priorityFeatures:
    for _m in RAT_feats["matches"]:
        if _m not in nomatch[_f].to_list():
            continue
        _no_match_subset_to_update = nomatch.filter(pl.col(_f) == _m)

        taxon_data_to_select_from = _collected_repeated_taxons.filter(
            pl.col("canonicalName") == _m
        ).select(["taxonID"] + priorityFeatures)
        selected_row = -1

        for i, _t in enumerate(taxon_data_to_select_from.iter_rows()):
            prev_col_index = priority_columns.index(
                _f
            )  # index of previous column of _f
            _x = _t[prev_col_index]
            if not bool(_x):
                selected_row = i
                break
        chosen_taxonId = taxon_data_to_select_from[selected_row, 0]

        _no_match_subset_to_update = _no_match_subset_to_update.with_columns(
            parentNameUsageID=pl.lit(chosen_taxonId)
        )
        print("-------\nFeature:", _f, ",Name:", _m)

        print(
            "taxon_data_to_select_from\n",
            taxon_data_to_select_from,
        )
        print("parentNameUsageID chosen:", chosen_taxonId)
        print("no_match_subset_to_update to update:")
        print(
            _no_match_subset_to_update.select(
                [
                    "taxonName",
                ]
                + priorityFeatures
            )
        )
        print("---------\n\n\n\n")
        updated_nomatch = pl.concat(
            [updated_nomatch, pl.LazyFrame(_no_match_subset_to_update)]
        )
        # all_taxon_data_to_be_selected_from = pl.concat((
        #     all_taxon_data_to_be_selected_from,taxon_data_to_select_from
        #     ))

updated_nomatch.write_csv("updated_nomatch.csv")
# all_taxon_data_to_be_selected_from.write_csv(
#     "all_taxon_data_to_be_selected_from.csv"
# )